In [21]:
# Global imports
import numpy as np
import random
import pandas as pd
import psutil
import os
import time
import glob
from typing import List, Tuple, Union, Optional

In [22]:
# Cria um array replicável
def gerar_array_replicavel(seed: int, tamanho: int) -> List[int]:
    random.seed(seed)
    vetor = list(range(tamanho))
    random.shuffle(vetor)
    return vetor

In [23]:
def calcular_aptidao_qap(
    solucao: Union[List[int], np.ndarray, List[List[int]]],
    matriz_fluxo: np.ndarray,
    matriz_distancia: np.ndarray
) -> int:
    solucao_flat: List[int] = []
    for x in solucao:
        if isinstance(x, (list, np.ndarray)):
            solucao_flat.extend(int(v) for v in x)
        else:
            solucao_flat.append(int(x))

    n: int = len(solucao_flat)
    custo: int = 0
    for i in range(n):
        for j in range(n):
            custo += matriz_fluxo[i][j] * matriz_distancia[solucao_flat[i]][solucao_flat[j]]
    return -custo

In [24]:
def inicializar_feromonio(n: int, valor_inicial: float = 1.0) -> np.ndarray:
    return np.full((n, n), valor_inicial)

In [25]:
def construir_solucao_qap(
    feromonio: np.ndarray,
    alpha: float,
    beta: float,
    heuristica: np.ndarray
) -> List[int]:
    """
    feromonio -> memória coletiva das soluções anteriores 
    heuristica -> informação local que guia a escolha 
    alpha -> peso do feromônio
    beta -> peso da heurística
    """
    n: int = feromonio.shape[0]
    nao_visitados: List[int] = list(range(n))
    solucao: List[int] = []

    while nao_visitados:
        if not solucao:
            atual: int = random.choice(nao_visitados)
        else:
            ultima: int = solucao[-1]
            pesos: List[float] = []
            for prox in nao_visitados:
                tau: float = feromonio[ultima][prox] ** alpha
                eta: float = heuristica[ultima][prox] ** beta
                pesos.append(tau * eta)
            pesos_np: np.ndarray = np.array(pesos)
            pesos_np /= pesos_np.sum()
            atual = random.choices(nao_visitados, weights=pesos_np)[0]

        solucao.append(atual)
        nao_visitados.remove(atual)

    return solucao

In [26]:
def busca_local(
    solucao: List[int],
    fluxo: np.ndarray,
    distancia: np.ndarray
) -> List[int]:
    melhor: List[int] = solucao.copy()
    melhor_custo: int = calcular_aptidao_qap(melhor, fluxo, distancia)
    melhorou: bool = True
    while melhorou:
        melhorou = False
        for i in range(len(solucao) - 1):
            for j in range(i + 1, len(solucao)):
                nova: List[int] = melhor.copy()
                nova[i], nova[j] = nova[j], nova[i]
                custo_novo: int = calcular_aptidao_qap(nova, fluxo, distancia)
                if custo_novo > melhor_custo:
                    melhor, melhor_custo = nova, custo_novo
                    melhorou = True
    return melhor

In [27]:
def atualizar_feromonio(
    feromonio: np.ndarray,
    solucoes: List[List[int]],
    custos: List[float],
    rho: float,
    Q: float
) -> np.ndarray:
    feromonio *= (1 - rho)  # evaporação
    for solucao, custo in zip(solucoes, custos):
        deposito = Q / max(custo, 1e-10)  # evita div/0
        for i in range(len(solucao) - 1):
            feromonio[solucao[i]][solucao[i + 1]] += deposito
        feromonio[solucao[-1]][solucao[0]] += deposito
    return feromonio

In [28]:
def aco_qap(
    fluxo: np.ndarray,
    distancia: np.ndarray,
    n_formigas: int = 10,
    alpha: float = 1,
    beta: float = 2,
    rho: float = 0.1,
    Q: float = 1,
    iter_max: int = 100
) -> Tuple[List[int], float]:
    n = len(fluxo)
    heuristica = 1 / (np.array(distancia) + 1e-10)
    feromonio = inicializar_feromonio(n)

    melhor_solucao: Optional[List[int]] = None
    melhor_custo: float = float("inf")

    for _ in range(iter_max):
        solucoes: List[List[int]] = []
        custos: List[float] = []
        for _ in range(n_formigas):
            s = construir_solucao_qap(feromonio, alpha, beta, heuristica)
            s = busca_local(s, fluxo, distancia)
            custo = calcular_aptidao_qap(s, fluxo, distancia)
            solucoes.append(s)
            custos.append(custo)
            if custo < melhor_custo:
                melhor_solucao, melhor_custo = s, custo

        feromonio = atualizar_feromonio(feromonio, solucoes, custos, rho, Q)

    if melhor_solucao is None:
        raise RuntimeError("ACO não encontrou nenhuma solução válida.")

    return melhor_solucao, melhor_custo

In [29]:
def ler_qap_com_n(caminho: str) -> Tuple[int, pd.DataFrame, pd.DataFrame]:
    with open(caminho, "r") as f:
        dados: List[int] = list(map(int, f.read().split()))

    n: int = dados[0]
    valores: List[int] = dados[1:]

    total_esperado: int = 2 * n * n
    if len(valores) != total_esperado:
        raise ValueError(f"Esperado {total_esperado} valores, mas encontrado {len(valores)}.")

    flow_flat: List[int] = valores[: n * n]
    dist_flat: List[int] = valores[n * n :]

    flow_df: pd.DataFrame = pd.DataFrame([flow_flat[i * n : (i + 1) * n] for i in range(n)])
    dist_df: pd.DataFrame = pd.DataFrame([dist_flat[i * n : (i + 1) * n] for i in range(n)])

    return n, flow_df, dist_df

In [ ]:
if __name__ == "__main__":
    arquivos = glob.glob("*.txt")

    for arquivo in arquivos:
        nome_instancia = os.path.splitext(os.path.basename(arquivo))[0]

        n, flow_df, dist_df = ler_qap_com_n(arquivo)
        matriz_fluxo = np.array(flow_df.values.tolist())
        matriz_distancia = np.array(dist_df.values.tolist())

        resultados = []

        for seed in range(42, 52):  # 42 até 51
            random.seed(seed)
            np.random.seed(seed)

            process = psutil.Process(os.getpid())
            tempo_inicio = time.time()

            # n_formigas = Número de soluções construídas por iteração
            # alpha = Influência do feromônio - > Se for alto, o algoritmo tende a explorar 
            # mais as soluções já conhecidas. Se for baixo, tende a explorar mais soluções novas.
            # beta = Influência da informação heurística
            # rho = Taxa de evaporação do feromônio
            # Q = Quantidade de feromônio depositada
            # iter_max = Número máximo de iterações
            parametros = {
                "n_formigas": random.randint(5, 20),
                "alpha": random.uniform(0.5, 2.0),
                "beta": random.uniform(1.0, 3.0),
                "rho": random.uniform(0.05, 0.2),
                "Q": 1,
                "iter_max": random.randint(50, 150)
            }

            melhor_solucao, melhor_custo = aco_qap(
                fluxo=matriz_fluxo,
                distancia=matriz_distancia,
                **parametros
            )

            tempo_fim = time.time()

            try:
                memoria_usada = process.memory_info().peak_wset / (1024 * 1024)  
            except AttributeError:
                memoria_usada = process.memory_info().rss / (1024 * 1024)  

            resultados.append({
                "instancia": nome_instancia,
                "seed": seed,
                "melhor_solucao": melhor_solucao,
                "custo": melhor_custo,
                "tempo_execucao_segundos": round(tempo_fim - tempo_inicio, 2),
                "memoria_usada_MB": round(memoria_usada, 2),
                **parametros
            })

            print(f"[{nome_instancia}] Seed {seed} finalizada. "
                  f"Custo: {melhor_custo} | Memória usada: {round(memoria_usada, 2)} MB")

        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_csv(f"resultados_{nome_instancia}_ACO.csv", index=False)

        print(f"\nResultados da instância {nome_instancia} salvos em resultados_{nome_instancia}_ACO.csv")


[Bur26a] Seed 42 finalizada. Custo: -5492156 | Memória usada: 131.7 MB


In [ ]:
solucao = aco_qap(matriz_fluxo, matriz_distancia, n_formigas=10, alpha=1, beta=2, rho=0.1, Q=1, iter_max=100)

In [ ]:
#calcular_aptidao_qap(solucao=solucao[0], matriz_fluxo=flow, matriz_distancia=dist)